# Project
playground for testing workflows

# Data set setup

In [1]:
import scipy.io

# prereqs import
import pandas as pd
import numpy as np
import math
import re
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [90]:
# load matlab data
mat = scipy.io.loadmat('matlab/emnist-digits')
mat.keys()
data = mat['dataset']


In [91]:
# see page 4 of the paper on emnist
# uncertain what 3rd entry is in dataset
X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

In [95]:
# For digits expect 280,000 total samples, 240,000 for training, 40,000 for testing. 
len(X_train)
len(y_train)

240000

In [ ]:
from sklearn import neural_network

testNN = neural_network.MLPClassifier()
testNN.fit(X_train,y_train)

In [ ]:
testNN.score(X_test,y_test)